```python
#!/usr/local/bin/python3
from Crypto.Cipher import AES
from Crypto.Util.number import long_to_bytes,bytes_to_long
import random
from secrets import token_bytes

flag = b'ictf{REDACTED}'


def helper(A,B,f,key) -> Tuple[ByteArray,ByteArray]:
    return (B,bytes([i ^ j for i,j in zip(A,f(B,key))]))
    
def func(A,key):
    c = AES.new(key,AES.MODE_ECB)
    out = c.encrypt(A)
    return out

seed = token_bytes(16)

def encrypt(message,keys):
    if len(message) > 32:
        raise Exception("no")
        
    A = message[:16]
    A += b'\x00' * (16 - len(A))
    B = message[16:]
    B += b'\x00' * (16 - len(B))
    
    
    for key in keys:
        (A,B) = helper(A,B,func,key)
    return A + B



def genkey():
    random.seed(seed)
    return long_to_bytes(random.randint(1 << 255, 1 << 256))
    
keys = [genkey() for _ in range(100)]


def main():

    while(True):
        try:
            option = int(input("1) print flag \n2) encrypt message\n>"))
            if (option == 1):
                print(bytes_to_long(encrypt(flag,keys)))
            elif (option == 2):
                print(bytes_to_long(encrypt(long_to_bytes(int(input('>'))),keys)))
            else:
                print('I don\'t understand that')
        except:
            print('dont try and break me >:C')
            exit(1)
if __name__ == '__main__':
    main()
```

# Checks

- challenge goal
    - Chosen plaintext attack to decipher msg0 (some stronger CPA game)
    - it 
- randomness
    - does not matter that much
    - deterministic 
- feistel network
    - decryption by swapping message isn't it?   

In [2]:
from Crypto.Cipher import AES
from Crypto.Util.number import long_to_bytes,bytes_to_long
import random
from secrets import token_bytes

flag = b'ictf{REDACTED}'


def helper(A,B,f,key):
    return (B,bytes([i ^ j for i,j in zip(A,f(B,key))]))
    
def func(A,key):
    c = AES.new(key,AES.MODE_ECB)
    out = c.encrypt(A)
    return out

seed = token_bytes(16)

def encrypt(message,keys):
    if len(message) > 32:
        raise Exception("no")
        
    A = message[:16]
    A += b'\x00' * (16 - len(A))
    B = message[16:]
    B += b'\x00' * (16 - len(B))
    
    
    for key in keys:
        (A,B) = helper(A,B,func,key)
    return A + B



def genkey():
    random.seed(seed)
    return long_to_bytes(random.randint(1 << 255, 1 << 256))
    
keys = [genkey() for _ in range(100)]

In [48]:
c = encrypt(b"12"*8 + b"34"*8,keys)
m = encrypt(c[16:] + c[:16],keys)
m = m[16:] + m[:16]
m

b'12121212121212123434343434343434'

$=>$ This is basically quite easy, as seen above. We only need to get the flag ciphertext, and send it reverse blocks again

[Wikipedia](https://en.wikipedia.org/wiki/Feistel_cipher):
> The round function can be made arbitrarily complicated, since it does not need to be designed to be invertible.[2]: 465  [3]: 347  Furthermore, the encryption and decryption operations are very similar, even identical in some cases, requiring only a reversal of the key schedule. Therefore, the size of the code or circuitry required to implement such a cipher is nearly halved. Unlike substitution-permutation networks, Feistel networks also do not depend on a substitution box that could cause timing side-channels in software implementations. 


In [49]:
from pwn import *

conn = remote("155.248.210.243", 42191)
conn.recvuntil(b">")         # wait for prompt
conn.sendline(b"1")          # choose option 1

response = conn.recvline(keepends=False)   # read flag output
print(response)
response = long_to_bytes(int(response))
response = response[16:] + response[:16] 
print(conn.recvuntil(b">"))        
conn.sendline(b"2")   
print(conn.recvuntil(b">"))       
conn.sendline(str(bytes_to_long(response)).encode()) 
response = conn.recvuntil(b"\n")          
response = long_to_bytes(int(response[:-1]))
response[16:] + response[:16]

[x] Opening connection to 155.248.210.243 on port 42191
[x] Opening connection to 155.248.210.243 on port 42191: Trying 155.248.210.243
[+] Opening connection to 155.248.210.243 on port 42191: Done
b'34705672679775921255229411981334655977384552999677796958609613547966093308352'
b'1) print flag \n2) encrypt message\n>'
b'>'


b'ictf{using_1_k3y_1n_f31573l_lol}'